## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns = ["EIN", "NAME"], axis = 1)

In [3]:
# Additional drops to remove noise and help improve performance of model
application_df = application_df.drop(columns = ["SPECIAL_CONSIDERATIONS", "STATUS"], axis = 1)

In [4]:
# Confirm Drop
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [5]:
# Determine the number of unique values in each column.
for column in application_df.columns:
    values = application_df[column].unique()
    print(f"{len(values)} unique values for {column} in application_df: {values}")
    print("\n")

17 unique values for APPLICATION_TYPE in application_df: ['T10' 'T3' 'T5' 'T7' 'T4' 'T6' 'T2' 'T9' 'T19' 'T8' 'T13' 'T12' 'T29'
 'T25' 'T14' 'T17' 'T15']


6 unique values for AFFILIATION in application_df: ['Independent' 'CompanySponsored' 'Family/Parent' 'National' 'Regional'
 'Other']


71 unique values for CLASSIFICATION in application_df: ['C1000' 'C2000' 'C3000' 'C1200' 'C2700' 'C7000' 'C7200' 'C1700' 'C4000'
 'C7100' 'C2800' 'C6000' 'C2100' 'C1238' 'C5000' 'C7120' 'C1800' 'C4100'
 'C1400' 'C1270' 'C2300' 'C8200' 'C1500' 'C7210' 'C1300' 'C1230' 'C1280'
 'C1240' 'C2710' 'C2561' 'C1250' 'C8000' 'C1245' 'C1260' 'C1235' 'C1720'
 'C1257' 'C4500' 'C2400' 'C8210' 'C1600' 'C1278' 'C1237' 'C4120' 'C2170'
 'C1728' 'C1732' 'C2380' 'C1283' 'C1570' 'C2500' 'C1267' 'C3700' 'C1580'
 'C2570' 'C1256' 'C1236' 'C1234' 'C1246' 'C2190' 'C4200' 'C0' 'C3200'
 'C5200' 'C1370' 'C2600' 'C1248' 'C6100' 'C1820' 'C1900' 'C2150']


5 unique values for USE_CASE in application_df: ['ProductDev' 'Preservation' '

In [6]:
# Look at APPLICATION_TYPE value counts for binning
application_type_dict = {}

for app in application_df["APPLICATION_TYPE"].unique():
    application_type_dict[app] = 0
    
for app in application_df["APPLICATION_TYPE"]:
    application_type_dict[app] += 1
    
application_type_dict

{'T10': 528,
 'T3': 27037,
 'T5': 1173,
 'T7': 725,
 'T4': 1542,
 'T6': 1216,
 'T2': 16,
 'T9': 156,
 'T19': 1065,
 'T8': 737,
 'T13': 66,
 'T12': 27,
 'T29': 2,
 'T25': 3,
 'T14': 3,
 'T17': 1,
 'T15': 2}

In [7]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
app_counts = list(application_type_dict.values())
application_types_to_replace = []
    
app_counts.sort(reverse = True)

for app in application_df["APPLICATION_TYPE"].unique():
    if application_type_dict[app] not in app_counts[0:9]:
        application_types_to_replace.append(app)

In [8]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [9]:
# Look at CLASSIFICATION value counts for binning
classification_dict = {}

for cls in application_df["CLASSIFICATION"].unique():
    classification_dict[cls] = 0
    
for cls in application_df["CLASSIFICATION"]:
    classification_dict[cls] += 1
    
classification_dict

{'C1000': 17326,
 'C2000': 6074,
 'C3000': 1918,
 'C1200': 4837,
 'C2700': 104,
 'C7000': 777,
 'C7200': 32,
 'C1700': 287,
 'C4000': 194,
 'C7100': 75,
 'C2800': 95,
 'C6000': 15,
 'C2100': 1883,
 'C1238': 10,
 'C5000': 116,
 'C7120': 18,
 'C1800': 15,
 'C4100': 6,
 'C1400': 34,
 'C1270': 114,
 'C2300': 32,
 'C8200': 11,
 'C1500': 16,
 'C7210': 7,
 'C1300': 58,
 'C1230': 36,
 'C1280': 50,
 'C1240': 30,
 'C2710': 3,
 'C2561': 1,
 'C1250': 14,
 'C8000': 20,
 'C1245': 1,
 'C1260': 3,
 'C1235': 9,
 'C1720': 6,
 'C1257': 5,
 'C4500': 1,
 'C2400': 6,
 'C8210': 1,
 'C1600': 5,
 'C1278': 10,
 'C1237': 9,
 'C4120': 1,
 'C2170': 1,
 'C1728': 1,
 'C1732': 1,
 'C2380': 1,
 'C1283': 1,
 'C1570': 1,
 'C2500': 1,
 'C1267': 2,
 'C3700': 1,
 'C1580': 1,
 'C2570': 1,
 'C1256': 2,
 'C1236': 1,
 'C1234': 2,
 'C1246': 2,
 'C2190': 1,
 'C4200': 1,
 'C0': 3,
 'C3200': 2,
 'C5200': 1,
 'C1370': 1,
 'C2600': 1,
 'C1248': 1,
 'C6100': 1,
 'C1820': 1,
 'C1900': 1,
 'C2150': 1}

In [10]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for key in classification_dict.keys():
    if classification_dict[key] > 1:
        print(f"{key}: {classification_dict[key]}")

C1000: 17326
C2000: 6074
C3000: 1918
C1200: 4837
C2700: 104
C7000: 777
C7200: 32
C1700: 287
C4000: 194
C7100: 75
C2800: 95
C6000: 15
C2100: 1883
C1238: 10
C5000: 116
C7120: 18
C1800: 15
C4100: 6
C1400: 34
C1270: 114
C2300: 32
C8200: 11
C1500: 16
C7210: 7
C1300: 58
C1230: 36
C1280: 50
C1240: 30
C2710: 3
C1250: 14
C8000: 20
C1260: 3
C1235: 9
C1720: 6
C1257: 5
C2400: 6
C1600: 5
C1278: 10
C1237: 9
C1267: 2
C1256: 2
C1234: 2
C1246: 2
C0: 3
C3200: 2


In [11]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
cls_counts = list(classification_dict.values())
classifications_to_replace = []
    
cls_counts.sort(reverse = True)

for cls in classification_dict.keys():
    if classification_dict[cls] not in cls_counts[0:14]:
        classifications_to_replace.append(cls)

In [12]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
Other      441
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df)

In [14]:
application_df.head()

,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,5000,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,108590,1,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,5000,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,0
3,6692,1,0,0,0,1,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,142590,1,0,0,0,1,0,0,0,0,...,1,0,0,0,1,0,0,0,0,0


In [15]:
# Split our preprocessed data into our features and target arrays
y = application_df["IS_SUCCESSFUL"]

X = application_df.drop(columns = ["IS_SUCCESSFUL"], axis = 1)
X.head()

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [17]:
##### Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train.columns)
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 8

# Add additional layer to help improve performance of model
hidden_nodes_layer3 = 8

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 408       
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
 dense_2 (Dense)             (None, 8)                 72        
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 561
Trainable params: 561
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Adjust learning_rate to help improve performance of model
opt = tf.keras.optimizers.Adam(learning_rate = 0.0008)

# Compile the model
nn.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [19]:
# Train the model
# Change epochs to 100
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 721us/step - loss: 0.6185 - accuracy: 0.6804
Epoch 2/100
804/804 [==============================] - 1s 700us/step - loss: 0.5637 - accuracy: 0.7233
Epoch 3/100
804/804 [==============================] - 1s 695us/step - loss: 0.5555 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 1s 692us/step - loss: 0.5522 - accuracy: 0.7317
Epoch 5/100
804/804 [==============================] - 1s 688us/step - loss: 0.5504 - accuracy: 0.7311
Epoch 6/100
804/804 [==============================] - 1s 701us/step - loss: 0.5490 - accuracy: 0.7329
Epoch 7/100
804/804 [==============================] - 1s 703us/step - loss: 0.5482 - accuracy: 0.7321
Epoch 8/100
804/804 [==============================] - 1s 690us/step - loss: 0.5477 - accuracy: 0.7320
Epoch 9/100
804/804 [==============================] - 1s 687us/step - loss: 0.5472 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 1s 700us/step - l

804/804 [==============================] - 1s 696us/step - loss: 0.5392 - accuracy: 0.7360
Epoch 80/100
804/804 [==============================] - 1s 704us/step - loss: 0.5388 - accuracy: 0.7363
Epoch 81/100
804/804 [==============================] - 1s 706us/step - loss: 0.5386 - accuracy: 0.7357
Epoch 82/100
804/804 [==============================] - 1s 712us/step - loss: 0.5387 - accuracy: 0.7374
Epoch 83/100
804/804 [==============================] - 1s 717us/step - loss: 0.5392 - accuracy: 0.7369
Epoch 84/100
804/804 [==============================] - 1s 711us/step - loss: 0.5387 - accuracy: 0.7359
Epoch 85/100
804/804 [==============================] - 1s 707us/step - loss: 0.5382 - accuracy: 0.7371
Epoch 86/100
804/804 [==============================] - 1s 713us/step - loss: 0.5384 - accuracy: 0.7367
Epoch 87/100
804/804 [==============================] - 1s 710us/step - loss: 0.5384 - accuracy: 0.7363
Epoch 88/100
804/804 [==============================] - 1s 701us/step - loss:

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5516 - accuracy: 0.7329 - 262ms/epoch - 978us/step
Loss: 0.551578938961029, Accuracy: 0.7329446077346802


In [21]:
# Export our model to HDF5 file
nn.save("Resources/AlphabetSoupCharity_Optimization.h5")